In [1]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using Theano backend.


In [3]:
textfile = "data/potter"
textfile = open(textfile).read().lower()
badtext = '/:;][{}-=+)(*&^%$#@!~1234567890'
textfile = textfile.translate(None, badtext)

In [4]:
def translation_dicts(textfile):
    chars = sorted(list(set(textfile)))
    char_to_num_dict = {}
    num_to_char_dict = {}
    num = 0
    for c in chars:
        char_to_num_dict[c] = num
        num_to_char_dict[num] = c
        num += 1
    return num_to_char_dict, char_to_num_dict, chars

In [5]:
num_to_char_dict, char_to_num_dict, chars = translation_dicts(textfile)

sequence_length = 50 #num of chars per observation
beg_index = 0
end_index = sequence_length
X_mat = []
y_vector = []

for i in range(len(textfile) - sequence_length):
    x_row = textfile[i:i+sequence_length]
    x_row = [char_to_num_dict[j] for j in x_row]
    y_val = textfile[i+sequence_length]  #the character you're predicting based on the previous 100
    y_val = char_to_num_dict[y_val]
    X_mat.append(x_row)
    y_vector.append(y_val)

y_vector = np.asarray(y_vector)


In [6]:
#convert y to one-hot encoding
y_mat = np_utils.to_categorical(y_vector)

num_patterns = len(X_mat)
num_vocab = len(chars)
X = np.reshape(X_mat, (num_patterns, sequence_length, 1))
X = X/float(num_vocab) #normalize X


In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))  #WE NEED TO GO DEEPER!!!!
model.add(Dropout(0.2))
model.add(Dense(y_mat.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Without further ado . . .

In [9]:
#fit the model (only doing this for a few epochs as an example. going to load a pre-trained model to save time here.)
model.fit(X, y_mat, batch_size=64, nb_epoch=5, callbacks=callbacks_list)  #increase num epoch once we get this working well

Epoch 1/5
21032/21032 [==============================] - 238s - loss: 3.0798   
Epoch 2/5
21032/21032 [==============================] - 242s - loss: 3.0534   
Epoch 3/5
21032/21032 [==============================] - 247s - loss: 2.9927   
Epoch 4/5
21032/21032 [==============================] - 227s - loss: 2.8593   
Epoch 5/5
21032/21032 [==============================] - 227s - loss: 2.7762   


In [19]:
#load dicts from the original data this was trained on
char_to_num_dict = {'\t': 0, '\n': 1, ' ': 2, '"': 3, "'": 4, ',': 5, '.': 6, '?': 7, '\\': 8, 'a': 9, 'c': 11, 'b': 10, 'e': 13, 'd': 12, 'g': 15, 'f': 14, 'i': 17, 'h': 16, 'k': 19, 'j': 18, 'm': 21, 'l': 20, 'o': 23, 'n': 22, 'q': 25, 'p': 24, 's': 27, 'r': 26, 'u': 29, 't': 28, 'w': 31, 'v': 30, 'y': 33, 'x': 32, 'z': 34}
num_to_char_dict = {0: '\t', 1: '\n', 2: ' ', 3: '"', 4: "'", 5: ',', 6: '.', 7: '?', 8: '\\', 9: 'a', 10: 'b', 11: 'c', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'k', 20: 'l', 21: 'm', 22: 'n', 23: 'o', 24: 'p', 25: 'q', 26: 'r', 27: 's', 28: 't', 29: 'u', 30: 'v', 31: 'w', 32: 'x', 33: 'y', 34: 'z'}
    
n_chars = len(textfile)
n_vocab = len(chars)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 600
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = textfile[i:i + seq_length]
    seq_out = textfile[i + seq_length]
    dataX.append([char_to_num_dict[char] for char in seq_in])
    dataY.append(char_to_num_dict[seq_out])

n_patterns = len(dataX)
#print "Total Patterns: ", n_patterns

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y_mat.shape[1], activation='softmax'))

# load the network weights
#filename = "/Users/jamesledoux/Desktop/weights-improvement-13-1.9537.hdf5"  #dante
#filename = "/Users/jamesledoux/Desktop/weights-improvement-31-1.4242.hdf5"  #willie
#filename = "/Users/jamesledoux/Desktop/shakespeare-1.3267.hdf5"  #willie
filename = weightfile = "data/rowling-1.2660.hdf5"
#filename = "/Users/jamesledoux/Desktop/weights-improvement-46-1.4188.hdf5"  #dante
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
# a random seed taken from the text
print "Seed:"
print "\"", ''.join([num_to_char_dict[value] for value in pattern]), "\""

# trying out a custom seed
#pattern = "                                                            to be or not to be that is the question,"
#print pattern
#pattern = [char_to_int[c] for c in pattern]

#generate characters
for i in range(300):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = num_to_char_dict[index]
    seq_in = [num_to_char_dict[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


Seed:
" ow my whistle, you kick off from the ground, hard," said
madam hooch. "keep your brooms steady, rise a few feet, and then come
straight back down by leaning forward slightly. on my whistle  three
 two "
but neville, nervous and jumpy and frightened of being left on the
ground, pushed off hard before the whistle had touched madam hooch's
lips.
"come back, boy" she shouted, but neville was rising straight up like a
cork shot out of a bottle  twelve feet  twenty feet. harry saw his
scared white face look down at the ground falling away, saw him gasp,
slip sideways off the broom and 
wham  a thud  "
coagon baterts, notter. thich tas a had to talking alother and thene he dound hear thought he sas standing uotil the tall. harry and hermione had fooward at the thire pieces and talked to to the last had mulled out of the coor and foog to to the gash tas a hood craging ar the tall. harry had reen in